In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

import warnings
# Filter out unwanted termination criteria warnings from modcma
warnings.filterwarnings(action = "ignore", message = "Termination criteria met.*")

In [195]:
def build_schedule(solved_per_problem, evals_per_problem, max_iters = 100, log = True):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem))
    expected_evals = np.zeros(len(solved_per_problem))
    
    schedule = []

    for _ in range(0, max_iters):
        # solve_per_eval = (solved_per_problem.mul(prior_likelihood, axis=0).mean(axis=0) / evals_per_problem.mul(prior_likelihood, axis=0).mean(axis=0))
        solve_per_eval = (solved_per_problem / evals_per_problem).mul(prior_likelihood, axis=0).mean(axis=0)
        idxmax = solve_per_eval.idxmax()

        schedule.append(idxmax)
        expected_evals += evals_per_problem[idxmax].mul(prior_likelihood, axis=0).values

        prior_likelihood = (1 - solved_per_problem[idxmax]).values * prior_likelihood

        if log: print(f"choose: {idxmax}, unsolved: {prior_likelihood.sum():.2f}, expected_evals: {expected_evals.mean():.2f}")
    
    return (schedule, prior_likelihood, expected_evals)

In [52]:
# solved_per_problem = pd.DataFrame([[0.5, 0.1], [0.1, 0.5]], index = ["A", "B"]).transpose()
# evals_per_problem = pd.DataFrame([[100, 10], [10, 100]], index = ["A", "B"]).transpose()

# build_schedule(solved_per_problem, evals_per_problem, max_iters=10, log = True)

In [53]:
def evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem))
    expected_evals = np.zeros(len(solved_per_problem))

    for algo in schedule:
        expected_evals += evals_per_problem[algo].mul(prior_likelihood, axis=0).values
        prior_likelihood = (1 - solved_per_problem[algo]).values * prior_likelihood

    return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]

In [162]:
dim = 10
files = Path(f"output-raw/{dim}D").glob("*/*.csv")

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

In [163]:
all_df = pd.concat(dfs, ignore_index=True)
all_df.head()

,log10_target,solved,evaluations,fid,iid,rep,dim,param_id
0,2.0,True,375,6,101,1,10,Powell
1,1.8,True,375,6,101,1,10,Powell
2,1.6,True,479,6,101,1,10,Powell
3,1.4,True,487,6,101,1,10,Powell
4,1.2,True,487,6,101,1,10,Powell


In [164]:
all_df.iid.max()

600

In [165]:
# just CMA-ES
# all_df = all_df[all_df["param_id"].isin(["default", "2L", "4L", "8L", "16L", "32L", "64L"])]

In [166]:
mean_evals = all_df.groupby(["log10_target", "fid", "param_id"], as_index = False).agg({ # , "iid"
    "evaluations": "mean", 
    "solved": "mean",
})

In [167]:
# all_df.groupby(["log10_target", "fid", "param_id", "solved"], as_index = False).agg({ # , "iid"
#     "evaluations": "mean",
# }).query("log10_target == -8.0 and param_id == 'Powell'").sort_values(["fid", "param_id", "solved"])

In [168]:
# mean_evals["spe"] = mean_evals["solved"] / mean_evals["evaluations"]
mean_evals[np.logical_and(mean_evals["fid"] == 24, mean_evals["log10_target"] == -8.0)]

,log10_target,fid,param_id,evaluations,solved
322,-8.0,24,16L,67072.32,0.0
323,-8.0,24,2L,14968.80,0.0
324,-8.0,24,32L,100253.44,0.0
325,-8.0,24,4L,24613.44,0.0
326,-8.0,24,64L,167531.52,0.0
327,-8.0,24,8L,43824.32,0.0
328,-8.0,24,L-BFGS-B,435.60,0.0
329,-8.0,24,Powell,1451.70,0.0
330,-8.0,24,SLSQP,305.94,0.0
331,-8.0,24,active,5532.18,0.0


In [169]:
# mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:.0f}-i{x.iid:.0f}-d5-t{x.log10_target}", axis=1)
mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:02}-d{dim:02}-t{x.log10_target:.1f}", axis=1)
mean_evals.head()

,log10_target,fid,param_id,evaluations,solved,problem_id
0,-8.0,1,16L,10504.442,1.0,f01-d10-t-8.0
1,-8.0,1,2L,2252.808,1.0,f01-d10-t-8.0
2,-8.0,1,32L,14603.544,1.0,f01-d10-t-8.0
3,-8.0,1,4L,3603.346,1.0,f01-d10-t-8.0
4,-8.0,1,64L,21001.238,1.0,f01-d10-t-8.0


In [170]:
mean_evals = mean_evals.drop(columns = ["log10_target", "fid"]) #, "iid"
mean_evals.head()

,param_id,evaluations,solved,problem_id
0,16L,10504.442,1.0,f01-d10-t-8.0
1,2L,2252.808,1.0,f01-d10-t-8.0
2,32L,14603.544,1.0,f01-d10-t-8.0
3,4L,3603.346,1.0,f01-d10-t-8.0
4,64L,21001.238,1.0,f01-d10-t-8.0


In [171]:
evals_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "evaluations")
evals_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,active,default,elitist,mirrored,orthogonal
problem_id,,,,,,,,,,,,,,
f01-d10-t-0.2,2510.336,514.318,3559.538,876.212,5272.574,1590.380,23.0,77.66,13.0,293.172,318.660,261.070,259.212,278.294
f01-d10-t-0.4,2717.214,558.580,3850.626,943.972,5696.468,1720.056,23.0,78.49,13.0,318.190,349.702,282.948,281.760,304.686
f01-d10-t-0.6,2909.766,601.396,4147.322,1014.652,6111.520,1840.140,23.0,78.99,13.0,344.022,380.188,306.732,305.034,331.132
f01-d10-t-0.8,3114.000,645.682,4422.568,1084.232,6509.408,1979.874,23.0,79.17,13.0,371.144,409.476,331.382,329.024,357.224
f01-d10-t-1.0,3320.302,692.792,4721.452,1152.052,6896.188,2108.106,23.0,79.39,13.0,399.868,440.092,355.258,351.168,382.786


In [172]:
solved_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "solved")
solved_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,active,default,elitist,mirrored,orthogonal
problem_id,,,,,,,,,,,,,,
f01-d10-t-0.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [173]:
print(solved_per_problem.loc[solved_per_problem.max(1) == 0].index.to_list())

['f04-d10-t-0.2', 'f04-d10-t-0.4', 'f04-d10-t-0.6', 'f04-d10-t-0.8', 'f04-d10-t-1.0', 'f04-d10-t-1.2', 'f04-d10-t-1.4', 'f04-d10-t-1.6', 'f04-d10-t-1.8', 'f04-d10-t-2.0', 'f04-d10-t-2.2', 'f04-d10-t-2.4', 'f04-d10-t-2.6', 'f04-d10-t-2.8', 'f04-d10-t-3.0', 'f04-d10-t-3.2', 'f04-d10-t-3.4', 'f04-d10-t-3.6', 'f04-d10-t-3.8', 'f04-d10-t-4.0', 'f04-d10-t-4.2', 'f04-d10-t-4.4', 'f04-d10-t-4.6', 'f04-d10-t-4.8', 'f04-d10-t-5.0', 'f04-d10-t-5.2', 'f04-d10-t-5.4', 'f04-d10-t-5.6', 'f04-d10-t-5.8', 'f04-d10-t-6.0', 'f04-d10-t-6.2', 'f04-d10-t-6.4', 'f04-d10-t-6.6', 'f04-d10-t-6.8', 'f04-d10-t-7.0', 'f04-d10-t-7.2', 'f04-d10-t-7.4', 'f04-d10-t-7.6', 'f04-d10-t-7.8', 'f04-d10-t-8.0', 'f04-d10-t0.0', 'f04-d10-t0.2', 'f04-d10-t0.4', 'f24-d10-t-0.6', 'f24-d10-t-0.8', 'f24-d10-t-1.0', 'f24-d10-t-1.2', 'f24-d10-t-1.4', 'f24-d10-t-1.6', 'f24-d10-t-1.8', 'f24-d10-t-2.0', 'f24-d10-t-2.2', 'f24-d10-t-2.4', 'f24-d10-t-2.6', 'f24-d10-t-2.8', 'f24-d10-t-3.0', 'f24-d10-t-3.2', 'f24-d10-t-3.4', 'f24-d10-t-3.6',

In [234]:
schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem, evals_per_problem, max_iters = 1000)

choose: SLSQP, unsolved: 772.00, expected_evals: 389.85
choose: mirrored, unsolved: 539.02, expected_evals: 2865.89
choose: SLSQP, unsolved: 532.93, expected_evals: 3038.74
choose: SLSQP, unsolved: 528.19, expected_evals: 3210.41
choose: 4L, unsolved: 406.57, expected_evals: 9508.16
choose: SLSQP, unsolved: 403.54, expected_evals: 9641.34
choose: SLSQP, unsolved: 400.86, expected_evals: 9774.12
choose: SLSQP, unsolved: 398.48, expected_evals: 9906.54
choose: SLSQP, unsolved: 396.36, expected_evals: 10038.65
choose: SLSQP, unsolved: 394.46, expected_evals: 10170.46
choose: SLSQP, unsolved: 392.74, expected_evals: 10302.03
choose: SLSQP, unsolved: 391.19, expected_evals: 10433.36
choose: SLSQP, unsolved: 389.78, expected_evals: 10564.48
choose: SLSQP, unsolved: 388.50, expected_evals: 10695.41
choose: SLSQP, unsolved: 387.32, expected_evals: 10826.17
choose: L-BFGS-B, unsolved: 385.58, expected_evals: 11017.96
choose: L-BFGS-B, unsolved: 383.95, expected_evals: 11209.38
choose: L-BFGS-B,

In [235]:
f"{schedule}"

"['SLSQP', 'mirrored', 'SLSQP', 'SLSQP', '4L', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '8L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'el

In [236]:
pd.value_counts(schedule)

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_84998/2913986251.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(schedule)
/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_84998/2913986251.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(schedule)


Powell      416
64L         264
32L         102
elitist      93
L-BFGS-B     75
mirrored     32
SLSQP        13
16L           3
4L            1
8L            1
Name: count, dtype: int64

In [237]:
penalty = 1_000_000 * dim

schedule_ert = evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty)
f"{schedule_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_84998/3395898693.py:11: RuntimeWarning: divide by zero encountered in divide
  return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'1_037_028.40'

In [238]:
mean_evals.param_id.unique()

array(['16L', '2L', '32L', '4L', '64L', '8L', 'L-BFGS-B', 'Powell',
       'SLSQP', 'active', 'default', 'elitist', 'mirrored', 'orthogonal'],
      dtype=object)

In [240]:
# ert_schedule = []

# for _ in range(0, 100):
#     best_schedule = []
#     best_perf = np.inf

#     for config in mean_evals.param_id.unique():
#         ext_schedule = ert_schedule.copy()
#         ext_schedule.extend([config])
#         perf = evaluate_schedule(solved_per_problem, evals_per_problem, ext_schedule, penalty)
#         if perf < best_perf:
#             best_perf = perf
#             best_schedule = ext_schedule
    
#     ert_schedule = best_schedule
    
#     print(f"{best_perf:_.2f}")

In [242]:
# ert_schedule

In [243]:
# sbs
all_ert = (evals_per_problem / solved_per_problem).clip(upper=penalty).mean()
all_ert.sort_values()

param_id
32L           1.201943e+06
16L           1.349483e+06
8L            1.719456e+06
64L           1.898419e+06
4L            2.000678e+06
2L            2.140084e+06
orthogonal    2.539933e+06
active        2.581933e+06
mirrored      2.603510e+06
default       2.845900e+06
elitist       3.433795e+06
L-BFGS-B      4.672970e+06
SLSQP         4.846449e+06
Powell        5.818368e+06
dtype: float64

In [244]:
sbs = min(all_ert)
f"{sbs:_.2f}"

'1_201_942.84'

In [245]:
# vbs
vbs = (evals_per_problem / solved_per_problem).clip(upper=penalty).apply(min, axis = 1).mean()
f"{vbs:_.2f}"

'869_106.28'

In [246]:
print(f"SBS-VBS Gap closed (Resubstitution): {100 - 100 * (schedule_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (Resubstitution): 49.55%


In [ ]:
penalty = 1_000_000 * dim

erts = []

# method = "sbs"
method = "normal"

for i in range(1, 25):
    print(f"Evaluating f{i:02}", end = "\r")
    filter_string = f"f{i:02}-"

    just_fi = [x for x in solved_per_problem.index if x.startswith(filter_string)]
    all_but_fi = [x for x in solved_per_problem.index if not x.startswith(filter_string)]

    if method == "sbs":
        all_ert = (evals_per_problem.loc[all_but_fi] / solved_per_problem.loc[all_but_fi]).clip(upper=penalty).mean()
        schedule = [all_ert.sort_values().index[0]]
        print(schedule)
    else:
        schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem.loc[all_but_fi], evals_per_problem.loc[all_but_fi], max_iters = 1000, log = False)
    
    ert = evaluate_schedule(solved_per_problem.loc[just_fi], evals_per_problem.loc[just_fi], schedule, penalty)
    erts.append(ert)

lopo_ert = np.mean(erts)
f"{lopo_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_84998/3395898693.py:11: RuntimeWarning: divide by zero encountered in divide
  return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_84998/3395898693.py:11: RuntimeWarning: divide by zero encountered in divide
  return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'1_250_692.93'

In [260]:
schedule

['SLSQP',
 'mirrored',
 'SLSQP',
 'SLSQP',
 '4L',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '8L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '32L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '32L',
 'L-BFGS-B',
 'L-BFGS

In [261]:
# erts_sbs = erts
((pd.DataFrame(erts) - pd.DataFrame(erts_sbs)) / pd.DataFrame(erts_sbs)) #.sum()
np.log10(erts).mean()

4.3590934901311185

In [262]:
print(f"SBS-VBS Gap closed (LOPO): {100 - 100 * (lopo_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (LOPO): -14.65%


In [253]:
pd.DataFrame(schedule).value_counts()

0       
Powell      454
64L         177
32L         163
elitist     109
L-BFGS-B     78
SLSQP        13
16L           3
4L            1
8L            1
mirrored      1
Name: count, dtype: int64

In [92]:
(solved_per_problem["Powell"] > 0).mean()

0.41911764705882354

In [98]:
((solved_per_problem / evals_per_problem)["32L"] > 0).sort_values()

problem_id
f4-d10-t-8.0     False
f24-d10-t-6.0    False
f24-d10-t-6.2    False
f24-d10-t-6.4    False
f24-d10-t-6.6    False
                 ...  
f16-d10-t1.6      True
f16-d10-t1.4      True
f16-d10-t1.2      True
f17-d10-t-0.6     True
f9-d10-t2.0       True
Name: 32L, Length: 1224, dtype: bool

In [99]:
(solved_per_problem / evals_per_problem)["32L"]

problem_id
f1-d10-t-0.2    0.000281
f1-d10-t-0.4    0.000260
f1-d10-t-0.6    0.000241
f1-d10-t-0.8    0.000226
f1-d10-t-1.0    0.000212
                  ...   
f9-d10-t1.2     0.000181
f9-d10-t1.4     0.000208
f9-d10-t1.6     0.000234
f9-d10-t1.8     0.000260
f9-d10-t2.0     0.000292
Name: 32L, Length: 1224, dtype: float64